### Seq2Seq Text Summarization

---

<b>Goal</b>: Train Seq2Seq algorithm for text summary 

In [1]:
text = '''
Głównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w Mainz. Akcja toczy się w roku 1602 roku gdzie wraz z resztą braci bierze udział w polowaniu na czarownice. Jednak niespodziewane spotkanie swojej przyrodniej siostry Anny Held, osądzanej o wykonywanie czarów, powoduje że Laymann odrzuca regułę klasztoru. Podstępnie zakrada się do biblioteki gdzie studiuje zakazane księgi. Zostaje przyłapany przez swojego nauczyciela brata Jakoba i uwięziony w lochu.
Główny bohater spotyka tam starszego człowieka Lugaida Vandroiy, który przedstawia mu się jako druid (Reach Out For The Light). Spotkany człowiek opowiada Gabrielowi o innym zagrożonym wymiarze – Avantasii. W zamian za pomoc w uratowaniu Avantasii druid obiecuje uratowanie Anny. Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy. Vandroiy używa go by przenieść Gabriela do równoległego świata.
W tym czasie biskup Mainz Johann Adam von Bicken, brat Jakob oraz rządca Falk von Kronberg są w drodze do Rzymu gdzie zamierzają spotkać się z papieżem Clemensem VIII (Glory of Rome). Niosą ze sobą także księgę odkrytą przez Gabriela. Wedle starożytnego zapisu wynika że księga jest ostatnią siódmą częścią pieczęci, która w całości daje właścicielowi absolutną wiedzę gdy tylko dostanie się on do wieży wyznaczającej środek Avantasii.
Gdy Gabriel dostaje się do równoległego świata jest powitany przez dwóch jego mieszkańców elfa Elderana oraz krasnoluda Regrina (Inside). Opowiadają mu o toczącej się wojnie przeciwko siłom zła oraz o planach papieża (Sign Of The Cross). Jeżeli papież użyje pieczęci połączenie między Avantasią a światem ludzi zostanie zamknięte, a oba światy dotkną straszliwe kataklizmy. Gabriel przybywa w momencie gdy Clemens VIII rozmawia z tajemniczym głosem dochodzącym z wieży. Zręcznemu Gabrielowi udaje się skraść pieczęć papieżowi po czym zanosi ją do miasta elfów (The Tower). Zdarzenie to kończy pierwszy album.
Jednak Gabriel nie jest usatysfakcjonowany. Chce dowiedzieć się więcej o świecie Avantasii dlatego Elderane wysyła nowicjusza do drzewa poznania. Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice). Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach. Kielich jest więzieniem dla ogromnej ilości torturowanych dusz, artefakt strzeżony jest także przez siejącą postrach bestię. Mimo niepowodzeń delfickich wypraw, Gabriel i Regrin powracają na ziemię by zmierzyć się z bestią. Przyjaciele znajdują kielich i przewracają go co umożliwia ucieczkę duszom. Przebudzona bestia zabija jednak krasnoluda, Gabrielowi udaje się uciec.
Gabriel wraca do Vandroiya, który czekał na niego. Druid spełnia obietnice, zakrada się do więzienia by uwolnić Annę. Jednak znajduje tam „przemienionego” brata Jakoba który także chce uwolnić Annę. Falk von Kronberg nakrywa ich i każe aresztować. Rozpoczęła się walka w której poległ Vandroiy raniony przez Kronberga, który później zostaje uśmiercony przez brata Jakoba. Anna ucieka by ponownie złączyć się z Gabrielem. Podążają wspólnie nieznaną drogą w przyszłość (Into The Unknown).
'''

### ***0. Load libraries***

In [66]:
# Text imports
import re
import string
from langdetect import detect

from tensorflow.keras import preprocessing

# NLTK imports
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.cluster.util import cosine_distance

# NetworkX for TextRang Algorithm
import networkx as nx

# Keras text ans sequence imports
from tensorflow.keras.preprocessing import text as kpt
from tensorflow.keras.preprocessing import sequence

# Misc libraries
import numpy as np
import operator
from random import choice
from typing import List
from functools import reduce
from IPython.core.display import display, HTML

In [67]:
vector = List[float] # custom list element

def cos_sim(v: vector, w: vector) -> float:
    '''
    Calculate cosine similarity between two vectors. Return 0 if both vectors
    contain zeroes.

    Arguments:
        v:  vector
        w:  vector
    
    Returns:
        cosine similarity
    '''
    if ((sum(v) == 0) or (sum(w) == 0)):
        return 0
    else:
        return 1 - cosine_distance(v, w)


In [68]:
def retrieve_stop_words():
    '''
    Load stop words based on detected language

    Argumenst:
        None

    Returns:
        set of stop words 
    '''
    from spacy.lang.en.stop_words import STOP_WORDS as STOP_WORDS_en
    from spacy.lang.pl.stop_words import STOP_WORDS as STOP_WORDS_pl
    lang = detect(text)

    if lang == 'en':
        STOP_WORDS = STOP_WORDS_en
    elif lang == 'pl':
        STOP_WORDS = STOP_WORDS_pl
        return STOP_WORDS

In [69]:
STOP_WORDS = retrieve_stop_words()

In [70]:
class Preprocessing(object):
    '''
    Text cleaning and sentence tokenization class.
    '''
    def __init__(self, text):
        self.text = text
        self.oryg = text

    def lower(self):
        self.text = self.text.lower() 
        return self.text
    
    def remove_punctuation(self):
        self.text = self.text.translate(self.text.maketrans('', '', string.punctuation.replace('.', '')))
        return self.text 
    
    def remove_stop_words(self):
        self.text = ' '.join([word for word in self.text.split() if word not in STOP_WORDS])
        return self.text
    
    def remove_digits(self):
        self.text = re.sub(r'[\d+]', '', self.text)
        return self.text
    
    def sentence_tokenize(self):
        self.text = sent_tokenize(self.text)
        self.text = [sent.replace('.','') for sent in self.text]
        return self.text
    
    def basic_pipeline(self):
        self.lower()
        self.remove_digits()
        self.remove_punctuation()
        self.remove_stop_words()
        self.sentence_tokenize()
        return self.text

    def __call__(self):
        return self.text

In [71]:
def prepare_text(text: str) -> tuple:
    '''
    Preprocess the text, split it into sentences and return modified
    results as well as the original text, both as lists (split based on
    sentences)

    Arguments:
        text: original text
    
    Return
        tuple containing preprocessed text (as list of sentences) and 
        origianl text (as list of sentences)

    '''
    # make a class object
    cleaned_text = Preprocessing(text) 
    
    # preprocess and tokenize text into sentences
    sentences = cleaned_text.basic_pipeline() 

    # tokenize original text into sentences
    raw_sentences = Preprocessing(text).sentence_tokenize() 
    return sentences, raw_sentences

In [72]:
sentences, raw_sentences = prepare_text(text)

In [73]:
sentences

['głównym bohaterem gabriel laymann nowicjusz klasztoru dominikanów mainz',
 'akcja toczy wraz resztą braci bierze udział polowaniu czarownice',
 'niespodziewane spotkanie swojej przyrodniej siostry anny held osądzanej wykonywanie czarów powoduje laymann odrzuca regułę klasztoru',
 'podstępnie zakrada biblioteki studiuje zakazane księgi',
 'zostaje przyłapany swojego nauczyciela brata jakoba uwięziony lochu',
 'główny bohater spotyka starszego człowieka lugaida vandroiy przedstawia druid reach out for the light',
 'spotkany człowiek opowiada gabrielowi innym zagrożonym wymiarze – avantasii',
 'zamian pomoc uratowaniu avantasii druid obiecuje uratowanie anny',
 'razem udaje uciec lochów breaking away czym vandroiy zabiera gabriela kamieniołomu ukryty portal łączący oba światy',
 'vandroiy używa przenieść gabriela równoległego świata',
 'czasie biskup mainz johann adam von bicken brat jakob rządca falk von kronberg drodze rzymu zamierzają spotkać papieżem clemensem glory of rome',
 'nios

In [74]:
special_tokens = ('<PAD>', '<START>', '<END>')

vocab = dict(zip(special_tokens, range(len(special_tokens))))
vocab

{'<PAD>': 0, '<START>': 1, '<END>': 2}

In [75]:
sentences = [special_tokens[1] + ' '+  sentence + ' ' + special_tokens[2] for sentence in sentences]

In [76]:
sentences

['<START> głównym bohaterem gabriel laymann nowicjusz klasztoru dominikanów mainz <END>',
 '<START> akcja toczy wraz resztą braci bierze udział polowaniu czarownice <END>',
 '<START> niespodziewane spotkanie swojej przyrodniej siostry anny held osądzanej wykonywanie czarów powoduje laymann odrzuca regułę klasztoru <END>',
 '<START> podstępnie zakrada biblioteki studiuje zakazane księgi <END>',
 '<START> zostaje przyłapany swojego nauczyciela brata jakoba uwięziony lochu <END>',
 '<START> główny bohater spotyka starszego człowieka lugaida vandroiy przedstawia druid reach out for the light <END>',
 '<START> spotkany człowiek opowiada gabrielowi innym zagrożonym wymiarze – avantasii <END>',
 '<START> zamian pomoc uratowaniu avantasii druid obiecuje uratowanie anny <END>',
 '<START> razem udaje uciec lochów breaking away czym vandroiy zabiera gabriela kamieniołomu ukryty portal łączący oba światy <END>',
 '<START> vandroiy używa przenieść gabriela równoległego świata <END>',
 '<START> czas

In [77]:
tokenizer = preprocessing.text.Tokenizer(
    num_words=50,
    lower = True,
    split = ' ',
    oov_token=None,

)

tokenizer.fit_on_texts(sentences)

In [78]:
vocab.update(tokenizer.word_index)

In [79]:
seq = tokenizer.texts_to_sequences(sentences)
X_train = preprocessing.sequence.pad_sequences(seq, maxlen = 15, padding = 'post', truncating='post')

In [84]:
del vocab['start']
del vocab['end']

In [85]:
vocab

{'<PAD>': 0,
 '<START>': 1,
 '<END>': 2,
 'gabriel': 3,
 'the': 4,
 'brata': 5,
 'jakoba': 6,
 'vandroiy': 7,
 'gabrielowi': 8,
 'avantasii': 9,
 'druid': 10,
 'udaje': 11,
 'gabriela': 12,
 'von': 13,
 'laymann': 14,
 'klasztoru': 15,
 'mainz': 16,
 'anny': 17,
 'zakrada': 18,
 'zostaje': 19,
 'opowiada': 20,
 'uciec': 21,
 'czym': 22,
 'oba': 23,
 'światy': 24,
 'równoległego': 25,
 'świata': 26,
 'falk': 27,
 'kronberg': 28,
 'of': 29,
 'pieczęci': 30,
 'wieży': 31,
 'krasnoluda': 32,
 'elderane': 33,
 'kielich': 34,
 'uwolnić': 35,
 'annę': 36,
 'głównym': 37,
 'bohaterem': 38,
 'nowicjusz': 39,
 'dominikanów': 40,
 'akcja': 41,
 'toczy': 42,
 'wraz': 43,
 'resztą': 44,
 'braci': 45,
 'bierze': 46,
 'udział': 47,
 'polowaniu': 48,
 'czarownice': 49,
 'niespodziewane': 50,
 'spotkanie': 51,
 'swojej': 52,
 'przyrodniej': 53,
 'siostry': 54,
 'held': 55,
 'osądzanej': 56,
 'wykonywanie': 57,
 'czarów': 58,
 'powoduje': 59,
 'odrzuca': 60,
 'regułę': 61,
 'podstępnie': 62,
 'bibliotek

In [49]:
X_train

array([[35, 36,  1, 12, 37, 13, 38, 14,  0,  0,  0,  0,  0,  0,  0],
       [39, 40, 41, 42, 43, 44, 45, 46, 47,  0,  0,  0,  0,  0,  0],
       [48, 49, 15, 12, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [17,  3,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 5,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [18,  6,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 7,  8, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 9, 19, 20,  5, 10, 21, 22,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 5, 10, 23, 24,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [14, 11, 25, 11, 26, 27,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [28, 29,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1, 23, 24, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [27,  2,  0,  0,  0,  0,  0

In [50]:
vocab

{'<PAD>': 0,
 'gabriel': 3,
 'the': 4,
 'brata': 5,
 'jakoba': 6,
 'vandroiy': 7,
 'gabrielowi': 8,
 'avantasii': 9,
 'druid': 10,
 'udaje': 11,
 'gabriela': 12,
 'von': 13,
 'laymann': 14,
 'klasztoru': 15,
 'mainz': 16,
 'anny': 17,
 'zakrada': 18,
 'zostaje': 19,
 'opowiada': 20,
 'uciec': 21,
 'czym': 22,
 'oba': 23,
 'światy': 24,
 'równoległego': 25,
 'świata': 26,
 'falk': 27,
 'kronberg': 28,
 'of': 29,
 'pieczęci': 30,
 'wieży': 31,
 'krasnoluda': 32,
 'elderane': 33,
 'kielich': 34,
 'uwolnić': 35,
 'annę': 36,
 'głównym': 37,
 'bohaterem': 38,
 'nowicjusz': 39,
 'dominikanów': 40,
 'akcja': 41,
 'toczy': 42,
 'wraz': 43,
 'resztą': 44,
 'braci': 45,
 'bierze': 46,
 'udział': 47,
 'polowaniu': 48,
 'czarownice': 49,
 'niespodziewane': 50,
 'spotkanie': 51,
 'swojej': 52,
 'przyrodniej': 53,
 'siostry': 54,
 'held': 55,
 'osądzanej': 56,
 'wykonywanie': 57,
 'czarów': 58,
 'powoduje': 59,
 'odrzuca': 60,
 'regułę': 61,
 'podstępnie': 62,
 'biblioteki': 63,
 'studiuje': 64,
 'z

NameError: name 'vocab' is not defined